# AeroMAPS model - dev

In [ ]:
# Libraries import
import sys
import os
import os.path as pth
sys.path.insert(0, '../..')
    
from lcav.lca_problem import *
from lcav.helpers import *
from lcav.io.configuration import *
from lcav.gui.plots import *
import matplotlib.pyplot as plt 

# Pretty print for Sympy
from sympy import init_printing
init_printing()

# Paths
#os.chdir('./notebook')
DATA_FOLDER = './data'
CONFIGURATION_FILE = pth.join(DATA_FOLDER, 'configuration_methodo_ei391.yaml')

In [ ]:
class ModelTemplate(AeromapsModel):
    def __init__(self, name="model_template", *args, **kwargs):
        super().__init__(name=name, *args, **kwargs)

    def compute(
        self,
        input_example: pd.Series = pd.Series(dtype="float64"),
    ) -> Tuple[
        pd.Series,
    ]:

        # Calculations
        output_example = fun(input_example)

        # Outputs
        self.df.loc[
            :, "output_example"
        ] = output_example
        
        return (
            output_example,
        )

In [ ]:
# Libraries import
import sys
import os.path as pth
sys.path.insert(0, '../..')
import lca_algebraic as agb
from lcav.io.configuration import LCAProblemConfigurator
DATA_FOLDER = './data'
CONFIGURATION_FILE = pth.join(DATA_FOLDER, 'configuration_methodo_ei391.yaml')


# AeroMAPS Model definition
class LifeCycleAssessment(AeromapsModel):
    def __init__(self, name="life_cycle_assessment", *args, **kwargs):
        super().__init__(name=name, *args, **kwargs)
        _, model, methods = LCAProblemConfigurator(CONFIGURATION_FILE).generate()
        # TODO: preLCAALgebraic here?
        self.model = model
        self.methods = methods
        self.parameters = dict()

    def compute(
        self,
        param_1: pd.Series = pd.Series(dtype="float64"),
        param_2: pd.Series = pd.Series(dtype="float64"),
        param_3: pd.Series = pd.Series(dtype="float64"),
    ) -> Tuple[pd.Series, ...]:  # Python 3.9+: use tuple instead of Tuple

        # Parameters assignment
        args = locals().copy()
        args.pop('self')
        args.pop('__class__')
        for key, val in args.items():
            parameters[key] = val
        
        # LCIA calculation
        res = agb.compute_impacts(
            self.model,
            self.methods,
            **parameters
        )

        # Outputs
        outputs_list = list()
        for cat in res.columns:
            temporal_impacts = pd.Series(res.loc[:, cat].values)
            self.df_lca.loc[:, cat] = temporal_impacts
            outputs_list.append(temporal_impacts)
        # TODO: add declaration of df_lca in AeroMapsModel class
        
        return outputs_list